In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import json
import os
from dotenv import load_dotenv
import time
from datetime import date, datetime, timedelta

In [6]:
# make a test curl request
load_dotenv() # Looks for a .env file in the current directory

api_key = os.getenv("FBI_KEY")


In [7]:
state = "UT"
url = f"https://api.usa.gov/crime/fbi/cde/agency/byStateAbbr/{state}?API_KEY={api_key}"
x = requests.get(url)
print(x.text)

{"IRON":[{"ori":"UT0110000","counties":"IRON","is_nibrs":true,"latitude":37.716972,"longitude":-113.0595,"state_abbr":"UT","state_name":"Utah","agency_name":"Iron County Sheriff's Office","agency_type_name":"County","nibrs_start_date":"2019-09-01"},{"ori":"UT0110100","counties":"IRON","is_nibrs":true,"latitude":37.677555,"longitude":-113.061646,"state_abbr":"UT","state_name":"Utah","agency_name":"Cedar City Police Department","agency_type_name":"City","nibrs_start_date":"2020-01-01"},{"ori":"UT0110300","counties":"IRON","is_nibrs":true,"latitude":37.882727,"longitude":-113.290059,"state_abbr":"UT","state_name":"Utah","agency_name":"Parowan Police Department","agency_type_name":"City","nibrs_start_date":"2019-09-01"},{"ori":"UT0110400","counties":"IRON","is_nibrs":true,"latitude":37.882727,"longitude":-113.290059,"state_abbr":"UT","state_name":"Utah","agency_name":"Southern Utah University","agency_type_name":"University or College","nibrs_start_date":"2011-10-01"},{"ori":"UT0110600","c

In [9]:
data = x.json()
rows = []
for county, agencies in data.items():
    for agency in agencies:
        agency['county'] = county
        rows.append(agency)

df = pd.DataFrame(rows)
df.head()

,ori,counties,is_nibrs,latitude,longitude,state_abbr,state_name,agency_name,agency_type_name,nibrs_start_date,county
0,UT0110000,IRON,True,37.716972,-113.059500,UT,Utah,Iron County Sheriff's Office,County,2019-09-01,IRON
1,UT0110100,IRON,True,37.677555,-113.061646,UT,Utah,Cedar City Police Department,City,2020-01-01,IRON
2,UT0110300,IRON,True,37.882727,-113.290059,UT,Utah,Parowan Police Department,City,2019-09-01,IRON
3,UT0110400,IRON,True,37.882727,-113.290059,UT,Utah,Southern Utah University,University or College,2011-10-01,IRON
4,UT0110600,IRON,True,37.882727,-113.290059,UT,Utah,Brian Head Police Department,City,2020-01-01,IRON


In [10]:
df.tail()

,ori,counties,is_nibrs,latitude,longitude,state_abbr,state_name,agency_name,agency_type_name,nibrs_start_date,county
138,UTLED0100,NOT SPECIFIED,True,40.667883,-111.924240,UT,Utah,Wildlife Resources,Other State Agency,2010-07-01,NOT SPECIFIED
139,UTUHP0000,NOT SPECIFIED,True,40.678660,-111.958020,UT,Utah,Utah Highway Patrol,State Police,2011-06-01,NOT SPECIFIED
140,UTDI01700,NOT SPECIFIED,True,40.467753,-113.123979,UT,Utah,Goshute Tribal,Tribal,2021-01-01,NOT SPECIFIED
141,UT0181200,"SALT LAKE, UTAH",True,40.525146,-111.862350,UT,Utah,Draper Police Department,City,2007-01-01,"SALT LAKE, UTAH"
142,UT0220500,"SUMMIT, WASATCH",True,40.872060,-110.968486,UT,Utah,Park City Police Department,City,2021-01-01,"SUMMIT, WASATCH"


In [13]:
with open("states.json", 'r') as f:
    states = json.load(f)
states = states['states'] # to get the list out of the json

with open("crime_abbr.json", 'r') as f:
    crimejson = json.load(f)
crime_abbrs = crimejson['offenses'] # to get the list out of the json
print(crime_abbrs)

['V', 'ASS', 'BUR', 'LAR', 'MVT', 'HOM', 'RPE', 'ROB', 'ARS', 'P']


In [15]:
start_time = "01-1985" # how far back the FBI data explorer goes
end_time = "10-2025" # last month

In [21]:
url = f"https://api.usa.gov/crime/fbi/cde/summarized/state/UT/V?from=08-2025&to={end_time}&API_KEY={api_key}"
# url = f"https://api.usa.gov/crime/fbi/cde/agency/byStateAbbr/{state}?API_KEY={api_key}"
response = requests.get(url)
resp_json = response.json()
rates = resp_json['offenses']['rates']

rows = []
for state, vals in rates.items():
    for date_str, rate in vals.items():
        rows.append({
            "state": state,
            "date": date_str,
            "rate": rate
        })

df = pd.DataFrame(rows)
# df["date"] = pd.to_datetime(df["date"], format="%m-%Y")
df.head()

,state,date,rate
0,Utah,08-2025,20.80
1,Utah,09-2025,18.75
2,Utah,10-2025,2.86
3,United States,08-2025,28.31
4,United States,09-2025,25.33


In [24]:
rates

{'Utah': {'08-2025': 20.8, '09-2025': 18.75, '10-2025': 2.86},
 'United States': {'08-2025': 28.31, '09-2025': 25.33, '10-2025': 21.59},
 'Utah Clearances': {'08-2025': 11.73, '09-2025': 9.51, '10-2025': 1.33},
 'United States Clearances': {'08-2025': 13.52,
  '09-2025': 11.8,
  '10-2025': 10.14}}

In [23]:
df.head(12)

,state,date,rate
0,Utah,08-2025,20.80
1,Utah,09-2025,18.75
2,Utah,10-2025,2.86
3,United States,08-2025,28.31
4,United States,09-2025,25.33
5,United States,10-2025,21.59
6,Utah Clearances,08-2025,11.73
7,Utah Clearances,09-2025,9.51
8,Utah Clearances,10-2025,1.33
9,United States Clearances,08-2025,13.52


In [20]:
df['state'].value_counts()

state
Utah                        490
United States               490
Utah Clearances             490
United States Clearances    490
Name: count, dtype: int64

In [ ]:
# loop to get all the data
for st in states:
    for crime in crime_abbrs:
        url = f"https://api.usa.gov/crime/fbi/cde/summarized/state/{st}/{crime}?from={start_time}&to={end_time}&API_KEY={api_key}"
        